In [19]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .appName("Project_Group19")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

#spark_context.setLogLevel("INFO")
spark_context.setLogLevel("ERROR")

ConnectionRefusedError: [Errno 111] Connection refused

In [14]:
#imports 
from pyspark import SparkContext, SparkConf

In [15]:
def tokenize_lines(lines):
    tokenized_lines = lines.map(lambda line: line.split('"'))
    return tokenized_lines


In [16]:
def split_tabular_data(lines):
    tabular_data = []

    for line in lines :
        csv_line = StringIO(line[0].replace('\t', ','))
        reader = csv.reader(csv_line, delimiter=',')
        tabular_data.extend(reader)

    # Convert the tabular data to RDD
    rdd = spark_session.sparkContext.parallelize(tabular_data)

    return rdd

In [17]:
def preprocess_data(lines):
    lowercased_lines = lines.map(lambda line: line.lower())
    tokenized_lines = tokenize_lines(lowercased_lines)
    return tokenized_lines

In [18]:
data = spark_context.textFile("hdfs://192.168.2.105:9000/user/ubuntu/amazon_reviews_us_Apparel_v1_00.tsv")
print(data.take(3))

ConnectionRefusedError: [Errno 111] Connection refused

In [7]:
tokenized_lines = preprocess_data(data)

In [8]:
mini_batch = tokenized_lines

In [9]:
mini_batch

PythonRDD[3] at RDD at PythonRDD.scala:53

In [11]:
from io import StringIO
import csv

current_rdd = mini_batch

# Collect data from the RDD
tabular_data = current_rdd.map(lambda line: line[0].split('\t')).collect()

# Convert the tabular data to RDD
rdd = spark_session.sparkContext.parallelize(tabular_data)

# Print the first 3 rows
for row in rdd.take(3):
    print(row)

ERROR:root:Exception while sending command.                        (3 + 2) / 15]
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4J

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr
from pyspark.sql.functions import length

header = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']


# Assuming spark_session is your SparkSession object and rdd is your RDD
# Convert RDD to DataFrame
df = spark_session.createDataFrame(rdd, schema=header)
print("----Schema----")
df.printSchema()

#add review length
df = df.withColumn("review_length", length(df["review_body"]))
# as it is string now
df = df.withColumn("total_votes_float", df["total_votes"].cast("float"))
df = df.withColumn("helpful_votes_float", df["helpful_votes"].cast("float"))
df.show()


# Calculate correlation
correlation = df.corr("review_length", "helpful_votes_float")

# Print the correlation
print("Correlation between review length and helpful votes:", correlation)
